In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
# import QUANTAXIS as QA
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from tqdm import tqdm_notebook
from scipy.stats import mstats

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

crsp fiscal resample

In [14]:
def calc_exf(df):
    ret = df.ret_p1
    dret = df.dret_p1
    exf = df.mv - df.mv.shift(1) * df.retx_p1
    exf_to_mv = exf / df.mv.shift(1)
    
    car1 = ((ret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1])
    car3 = ((ret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1])
    car5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    wr5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) / \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    
#     ret_n1 = ((ret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n2 = ((ret[::-1]).rolling(2, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n3 = ((ret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n4 = ((ret[::-1]).rolling(4, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n6 = ((ret[::-1]).rolling(6, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n7 = ((ret[::-1]).rolling(7, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n8 = ((ret[::-1]).rolling(8, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    res = pd.DataFrame({
        'date':df.date,
        'mv':df.mv,
        'retx_p1':df.retx_p1,
        'exf':exf,
        'exf_to_mv':exf_to_mv,
        'car1':car1.shift(-1),
        'car3':car3.shift(-1),
        'car5':car5.shift(-1),
        'wr5':wr5.shift(-1),
#         'ret_n1':ret_n1.shift(-1),
#         'ret_n2':ret_n2.shift(-1),
#         'ret_n3':ret_n3.shift(-1),
#         'ret_n4':ret_n4.shift(-1),
#         'ret_n5':ret_n5.shift(-1),
#         'ret_n6':ret_n6.shift(-1),
#         'ret_n7':ret_n7.shift(-1),
#         'ret_n8':ret_n8.shift(-1)
    })
    return res

In [5]:
crsp_fa = pd.read_csv('data/crsp_fa_filtered.zip',
                     parse_dates=['date'], infer_datetime_format=True)

In [6]:
crsp_fa.head()

,date,permno,ret_p1,retx_p1,dret_p1,mv,mv_adj
0,1986-10-31,10000,0.178571,0.178571,1.059979,3002.34375,5796.726244
1,1986-06-30,10001,1.032450,1.000000,1.252963,6033.12500,12067.131553
2,1987-06-30,10001,1.023885,0.959184,1.018842,5822.12500,11205.083735
3,1988-06-30,10001,1.140535,1.063829,0.827947,6200.00000,11459.064979
4,1989-06-30,10001,1.199811,1.120000,1.005611,7007.00000,12326.466763


some explain

In [7]:
# crsp_fa.query('permno == 77418')
# test = crsp_fa.query('permno == 77418').set_index('date')
# test['2002':'2006']
# test['2002':'2006'].ret_p1

# p = 1
# for i in test['2002':'2006'].ret_p1:
#     p = i * p
# print(p)

# test['2002':'2006'].ret_p1.prod()
# test['2002':'2006'].dret_p1.prod()
# wr5_test = test['2002':'2006'].ret_p1.prod() / test['2002':'2006'].dret_p1.prod()
# wr5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) / \
#         ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])

In [8]:
# s = pd.Series(range(1,11))
# s
# s[::-1]
# s.rolling(5).sum()
# s[::-1].rolling(5).sum()
# s[::-1].rolling(5).sum()[::-1]
# s[::-1].rolling(5).sum()[::-1].shift(-1)
# s[::-1].rolling(5, min_periods=1).sum()[::-1].shift(-1)
# s.rolling(5, min_periods=1).sum()
# s.rolling(5, min_periods=1).apply(np.prod, raw=True)

In [9]:
# test.ret_p1
# test.ret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)
# # test.ret_p1.rolling(5, min_periods=1).apply(np.prod, raw=True)
# test.ret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)
# # test.ret_p1.rolling(5, min_periods=1).apply(np.prod, raw=True)
# test.dret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)
# (test.ret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)) / \
# (test.dret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1))
# (test.ret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)) / \
# (test.dret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1))
# (test.ret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)) - \
# (test.dret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1))

验证mv adjust是否正确

In [7]:
# cpi = pd.read_excel('data/CPIAUCSL.xlsx', index_col=0)
# crsp_fa = crsp_fa.set_index('date')
# (crsp_fa['1986-10-31'].query('permno == 10000').mv.iloc[0] / \
# cpi['1986-11'].cpi_adjust.iloc[0]) - \
# (crsp_fa['1986-10-31'].query('permno == 10000').mv_adj.iloc[0])

查看firm-years数量

ps:因为流通股数的单位默认是1000, 故mv_adj只需要大于50000即可

In [24]:
# crsp_fa[crsp_fa.mv_adj >= 5e+4].shape[0]
# crsp_fa = crsp_fa.reset_index().set_index('date')[:'2008']
# crsp_fa[(crsp_fa.mv_adj.shift(1) * crsp_fa.retx_p1) >= 5e+4].shape[0]

假设之一:直接把交易前小市值筛掉的做法是错误的

In [8]:
# 这里应该是用到08年的值去筛选出permno,即筛选出在08年pre transaction market capitalization大于50million的公司
# crsp_filter = crsp_fa.reset_index().set_index('date')[:'2008']
crsp_filter = crsp_fa.set_index('date')[:'2008']

In [9]:
filter_permno = crsp_filter[(crsp_filter.mv_adj.shift(1) * crsp_filter.retx_p1) >= 5e+4].\
permno.unique().tolist()

In [10]:
crsp_fa_filtered = crsp_fa.set_index('permno').loc[filter_permno].reset_index()

In [12]:
# 筛选完公司在筛选一遍市值
crsp_fa_filtered = crsp_fa_filtered[(crsp_fa_filtered.mv_adj.shift(1) * crsp_fa_filtered.retx_p1) >= 5e+4]

In [15]:
tic = time.perf_counter()
crsp_fa_filtered = crsp_fa_filtered.set_index('permno').groupby('permno').apply(calc_exf)
toc = time.perf_counter()
print(toc-tic)

137.3940894729472


In [16]:
crsp_fa1 = crsp_fa_filtered.copy()
crsp_fa1 = crsp_fa1.dropna(subset=['exf_to_mv']) # 只是剔除exf_to_mv中的异常值
# crsp_fa1 = crsp_fa1.dropna() # 只要有异常值，就全部剔除，两者差距不是很大
crsp_fa1 = crsp_fa1.reset_index().set_index('date')[:'2008'] # 筛选出到08年的数据
# crsp_fa1 = crsp_fa1.loc['1973':'2001']
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.05, 0.05]) # 这里也可以调整

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

permno  exf_to_mv      car1      car3      car5  \
exf_to_mv                                                                      
(-1.119, -0.0433]      57378.352400  -0.095315  0.152169  0.376332  0.586771   
(-0.0433, -0.00665]    56140.624542  -0.021030  0.051632  0.155780  0.261956   
(-0.00665, -1.29e-07]  55815.858470  -0.001450  0.050623  0.153968  0.276594   
(-1.29e-07, 0.000949]  54901.221136   0.000194  0.059818  0.167783  0.286825   
(0.000949, 0.00467]    54912.848611   0.002656  0.076329  0.184254  0.284141   
(0.00467, 0.0111]      55785.594687   0.007579  0.064765  0.166630  0.273732   
(0.0111, 0.0243]       56901.330889   0.016814  0.050035  0.160993  0.262499   
(0.0243, 0.0618]       58469.123034   0.039185  0.059337  0.161812  0.226974   
(0.0618, 0.197]        58934.711806   0.114662  0.031702  0.091249  0.146200   
(0.197, 0.429]         62004.157174   0.360115  0.017873  0.013938  0.068475   

                            wr5  
exf_to_mv                        
(-1.119, -0.0433]      1.337995  
(-0.0433, -0.00665]    1.164934  
(-0.00665, -1.29e-07]  1.164484  
(-1.29e-07, 0.000949]  1.170294  
(0.000949, 0.00467]    1.160293  
(0.00467, 0.0111]      1.146662  
(0.0111, 0.0243]       1.151251  
(0.0243, 0.0618]       1.128739  
(0.0618, 0.197]        1.081717  
(0.197, 0.429]         1.033058

假设之二，即我不做任何市值筛选，还是会得到和paper中一样的结论

In [17]:
crsp_fa_filtered = crsp_fa.copy()

tic = time.perf_counter()
crsp_fa_filtered = crsp_fa_filtered.set_index('permno').groupby('permno').apply(calc_exf)
toc = time.perf_counter()
print(toc-tic)

crsp_fa1 = crsp_fa_filtered.copy()
crsp_fa1 = crsp_fa1.dropna(subset=['exf_to_mv']) # 只是剔除exf_to_mv中的异常值
# crsp_fa1 = crsp_fa1.dropna() # 只要有异常值，就全部剔除，两者差距不是很大
crsp_fa1 = crsp_fa1.reset_index().set_index('date')[:'2008'] # 筛选出到08年的数据
# crsp_fa1 = crsp_fa1.loc['1973':'2001']
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.05, 0.05]) # 这里也可以调整

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

182.2740446329069


,permno,exf_to_mv,car1,car3,car5,wr5
exf_to_mv,,,,,,
"(-1.066, -0.0255]",56226.456810,-0.054046,0.045806,0.141582,0.203472,1.139498
"(-0.0255, -0.000856]",55974.623503,-0.010222,0.035799,0.103337,0.187523,1.114622
"(-0.000856, 1.41e-08]",53734.183104,-0.000061,0.033741,0.056033,0.083093,1.085453
"(1.41e-08, 0.000302]",54156.712786,0.000026,0.030298,0.036698,0.020392,1.055692
"(0.000302, 0.0035]",55674.624136,0.001690,0.036124,0.064933,0.074157,1.045693
"(0.0035, 0.00982]",56280.908184,0.006302,0.029131,0.045916,0.058025,1.038979
"(0.00982, 0.0234]",57164.923065,0.015595,0.018022,0.024310,0.041087,1.030724
"(0.0234, 0.0624]",58528.869558,0.038883,0.011559,0.018660,-0.032688,0.989066
"(0.0624, 0.207]",59355.905493,0.118685,-0.016546,-0.076747,-0.150347,0.924941


In [18]:
crsp_fa1.shape

(189510, 9)

筛选公司不会改变sample的分布，但是筛选市值会改变其分布

筛选市值大于50million和剔除小于50million的公司是两种不同的处理方式

In [19]:
crsp_filter = crsp_fa.set_index('date')[:'2008']
filter_permno = crsp_filter[(crsp_filter.mv_adj.shift(1) * crsp_filter.retx_p1) >= 5e+4].\
permno.unique().tolist()

crsp_fa_filtered = crsp_fa.set_index('permno').loc[filter_permno].reset_index()

tic = time.perf_counter()
crsp_fa_filtered = crsp_fa_filtered.set_index('permno').groupby('permno').apply(calc_exf)
toc = time.perf_counter()
print(toc-tic)

crsp_fa1 = crsp_fa_filtered.copy()
crsp_fa1 = crsp_fa1.dropna(subset=['exf_to_mv']) # 只是剔除exf_to_mv中的异常值
# crsp_fa1 = crsp_fa1.dropna() # 只要有异常值，就全部剔除，两者差距不是很大
crsp_fa1 = crsp_fa1.reset_index().set_index('date')[:'2008'] # 筛选出到08年的数据
# crsp_fa1 = crsp_fa1.loc['1973':'2001']
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.05, 0.05]) # 这里也可以调整

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

158.79174762379853


,permno,exf_to_mv,car1,car3,car5,wr5
exf_to_mv,,,,,,
"(-1.066, -0.0261]",56231.114253,-0.054192,0.053575,0.174042,0.261691,1.163329
"(-0.0261, -0.00114]",56067.244276,-0.010753,0.043743,0.125188,0.237322,1.134676
"(-0.00114, 5.6e-08]",53933.997280,-0.000092,0.046179,0.116728,0.191206,1.128983
"(5.6e-08, 0.000466]",54328.340399,0.000060,0.046644,0.099444,0.137528,1.100556
"(0.000466, 0.00379]",55713.870438,0.001938,0.038457,0.072648,0.101784,1.056119
"(0.00379, 0.0102]",56242.428814,0.006644,0.036466,0.069184,0.087717,1.054752
"(0.0102, 0.0236]",57251.515529,0.015937,0.023408,0.045566,0.079703,1.048517
"(0.0236, 0.0625]",58577.015189,0.039091,0.021788,0.043701,0.004793,1.008651
"(0.0625, 0.205]",59350.633530,0.117770,-0.007937,-0.048625,-0.113072,0.945136


In [20]:
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).count().exf_to_mv

exf_to_mv
(-1.066, -0.0261]      17645
(-0.0261, -0.00114]    17644
(-0.00114, 5.6e-08]    17644
(5.6e-08, 0.000466]    17644
(0.000466, 0.00379]    17644
(0.00379, 0.0102]      17644
(0.0102, 0.0236]       17644
(0.0236, 0.0625]       17644
(0.0625, 0.205]        17644
(0.205, 0.453]         17644
Name: exf_to_mv, dtype: int64

In [36]:
crsp_filter = crsp_fa.set_index('date')[:'2008']
filter_permno_reverse = crsp_filter[(crsp_filter.mv_adj.shift(1) * crsp_filter.retx_p1) < 5e+4].\
permno.unique().tolist()

filter_permno = list(set(crsp_fa.permno.unique().tolist()) - set(filter_permno_reverse))

crsp_fa_filtered = crsp_fa.set_index('permno').loc[filter_permno]#.reset_index()

In [37]:
tic = time.perf_counter()
# crsp_fa_filtered = crsp_fa_filtered.set_index('permno').groupby('permno').apply(calc_exf)
crsp_fa_filtered = crsp_fa_filtered.groupby('permno').apply(calc_exf)
toc = time.perf_counter()
print(toc-tic)

crsp_fa1 = crsp_fa_filtered.copy()
crsp_fa1 = crsp_fa1.dropna(subset=['exf_to_mv']) # 只是剔除exf_to_mv中的异常值
# crsp_fa1 = crsp_fa1.dropna() # 只要有异常值，就全部剔除，两者差距不是很大
crsp_fa1 = crsp_fa1.reset_index().set_index('date')[:'2008'] # 筛选出到08年的数据
# crsp_fa1 = crsp_fa1.loc['1973':'2001']
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.05, 0.05]) # 这里也可以调整

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

ValueError: cannot reindex from a duplicate axis

In [ ]:
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).count().exf_to_mv

exf_to_mv : $EXF_{t}/MV_{t-1}$

In [14]:
crsp_fa1 = crsp_fa_filtered.copy()
crsp_fa1 = crsp_fa1.dropna(subset=['exf_to_mv']) # 只是剔除exf_to_mv中的异常值
# crsp_fa1 = crsp_fa1.dropna() # 只要有异常值，就全部剔除，两者差距不是很大
crsp_fa1 = crsp_fa1.reset_index().set_index('date')[:'2008'] # 筛选出到08年的数据
# crsp_fa1 = crsp_fa1.loc['1973':'2001']
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.05, 0.05]) # 这里也可以调整

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

,permno,exf_to_mv,car1,car3,car5,wr5
exf_to_mv,,,,,,
"(-1.066, -0.0261]",56231.114253,-0.054192,0.053575,0.174042,0.261691,1.163329
"(-0.0261, -0.00114]",56067.244276,-0.010753,0.043743,0.125188,0.237322,1.134676
"(-0.00114, 5.6e-08]",53933.997280,-0.000092,0.046179,0.116728,0.191206,1.128983
"(5.6e-08, 0.000466]",54328.340399,0.000060,0.046644,0.099444,0.137528,1.100556
"(0.000466, 0.00379]",55713.870438,0.001938,0.038457,0.072648,0.101784,1.056119
"(0.00379, 0.0102]",56242.428814,0.006644,0.036466,0.069184,0.087717,1.054752
"(0.0102, 0.0236]",57251.515529,0.015937,0.023408,0.045566,0.079703,1.048517
"(0.0236, 0.0625]",58577.015189,0.039091,0.021788,0.043701,0.004793,1.008651
"(0.0625, 0.205]",59350.633530,0.117770,-0.007937,-0.048625,-0.113072,0.945136


In [35]:
crsp_fa1.shape[0]

163602

Significant issuance & Significant repurchase

EXF is greater than 5% of pre-issuance market capitalization

$EXF_t >= 0.05 * (1+rx_t) *MV_{t-1}$

equals

$EXF_t / MV_{t-1} >= 0.05 * (1+rx_t)$

In [26]:
crsp_fa1[crsp_fa1.exf_to_mv > (0.05 * crsp_fa1.retx_p1)].describe().loc[['count', 'mean']]

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
count,38481.000000,3.848100e+04,38481.000000,38481.000000,38481.000000,35540.000000,35540.00000,35540.000000,35540.000000
mean,61160.977469,1.014590e+06,1.266222,196256.077069,0.366409,-0.011391,-0.09871,-0.181667,0.892571


In [27]:
crsp_fa1[crsp_fa1.exf_to_mv <= (-0.05 * crsp_fa1.retx_p1)].describe().loc[['count', 'mean']]

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
count,11216.000000,1.121600e+04,11216.000000,11216.000000,11216.000000,10374.000000,10374.000000,10374.000000,10374.000000
mean,57225.569633,1.619939e+06,1.107248,-231351.158139,-0.129547,0.063426,0.210477,0.311944,1.184296


In [28]:
crsp_fa1[(crsp_fa1.exf_to_mv >(-0.05 * crsp_fa1.retx_p1)) & \
         (crsp_fa1.exf_to_mv <= (0.05 * crsp_fa1.retx_p1))].describe().loc[['count', 'mean']]

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
count,126744.000000,1.267440e+05,126744.00000,126744.000000,126744.000000,117688.000000,117688.000000,117688.000000,117688.000000
mean,55665.221446,1.365509e+06,1.11561,-2135.690875,0.005052,0.035517,0.084474,0.127266,1.082469


long Significant repurchase,  short Significant issuance

In [29]:
crsp_fa1[crsp_fa1.exf_to_mv <= (-0.05 * crsp_fa1.retx_p1)].describe().loc[['mean']] - \
crsp_fa1[crsp_fa1.exf_to_mv > (0.05 * crsp_fa1.retx_p1)].describe().loc[['mean']]

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
mean,-3935.407837,605348.659812,-0.158974,-427607.235208,-0.495956,0.074817,0.309187,0.493611,0.291725


查看样本的个数

In [40]:
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).count().exf_to_mv

exf_to_mv
(-1.066, -0.0261]      17645
(-0.0261, -0.00114]    17644
(-0.00114, 5.6e-08]    17644
(5.6e-08, 0.000466]    17644
(0.000466, 0.00379]    17644
(0.00379, 0.0102]      17644
(0.0102, 0.0236]       17644
(0.0236, 0.0625]       17644
(0.0625, 0.205]        17644
(0.205, 0.453]         17644
Name: exf_to_mv, dtype: int64

In [41]:
crsp_fa_filtered.head()

,date,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
permno,,,,,,,,,
10002,1986-12-31,14540.625,1.064515,NaN,NaN,-0.032335,-0.574328,-0.585850,0.452834
10002,1987-12-31,11750.000,0.808082,-0.013582,-9.341005e-07,-0.335878,-0.376648,-0.885846,0.503963
10002,1988-12-31,9840.625,0.837500,0.004930,4.195653e-07,-0.352979,-0.369345,-0.494632,0.759094
10002,1989-12-31,7196.875,0.731343,0.006905,7.017218e-07,0.127311,0.065773,0.458322,1.260865
10002,1990-12-31,5875.000,0.816327,-0.002175,-3.021710e-07,-0.253274,-0.137564,-0.538593,0.846217


In [47]:
sample = crsp_fa_filtered.loc[77418].reset_index().set_index('date')

In [50]:
sample

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
date,,,,,,,,,
1992-06-30,77418,7.399350e+04,0.885246,NaN,NaN,1.485581,11.499109,30.486357,13.248084
1993-06-30,77418,2.180040e+05,2.740738,1.520721e+04,0.205521,0.494495,7.960438,43.397940,17.811951
1994-06-30,77418,4.129650e+05,1.540542,7.712078e+04,0.353759,1.913570,5.912793,121.919548,41.619246
1995-06-30,77418,1.652420e+06,3.087717,3.773010e+05,0.913639,0.768163,7.564375,35.965321,13.885897
1996-06-30,77418,3.935138e+06,1.988631,6.490840e+05,0.392808,-0.050407,18.249801,16.001558,9.100687
1997-06-30,77418,5.443073e+06,1.271430,4.398232e+05,0.111768,2.520221,13.598321,4.597881,4.922155
1998-06-30,77418,2.272824e+07,3.822978,1.919487e+06,0.352648,2.969190,2.551360,0.903554,1.772812
1999-06-30,77418,1.195158e+08,4.185493,2.438690e+07,1.072978,-0.133864,-0.379197,-0.537914,0.498832
2000-06-30,77418,1.216507e+08,0.957955,7.159994e+06,0.059908,0.058645,-0.363463,-0.549944,0.478177


In [48]:
wt1 = sample.exf * (1 - sample.wr5)

In [49]:
wt1

date
1992-06-30             NaN
1993-06-30   -2.556628e+05
1994-06-30   -3.132588e+06
1995-06-30   -4.861862e+06
1996-06-30   -5.258027e+06
1997-06-30   -1.725055e+06
1998-06-30   -1.483403e+06
1999-06-30    1.222192e+07
2000-06-30    3.736253e+06
2001-12-31    1.249860e+07
2002-12-31    1.916245e+05
2003-12-31    3.433335e+05
2004-12-31    3.645494e+05
2005-12-31    5.976083e+05
2006-12-31   -2.049306e+06
2007-12-31    2.347058e+06
2008-12-31    1.600868e+05
2009-12-31    6.733572e+06
2010-12-31    1.024319e+06
2011-12-31    1.316291e+06
2012-12-31    3.594415e+05
2013-12-31             NaN
dtype: float64

In [52]:
sample = sample.assign(wt1 = np.where(sample.exf > 0, sample.exf * (1 - sample.wr5), 
         sample.exf * (1 - sample.wr5) * sample.mv / (sample.mv.shift(1) * sample.retx_p1)))

In [53]:
sample

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,wt1
date,,,,,,,,,,
1992-06-30,77418,7.399350e+04,0.885246,NaN,NaN,1.485581,11.499109,30.486357,13.248084,NaN
1993-06-30,77418,2.180040e+05,2.740738,1.520721e+04,0.205521,0.494495,7.960438,43.397940,17.811951,-2.556628e+05
1994-06-30,77418,4.129650e+05,1.540542,7.712078e+04,0.353759,1.913570,5.912793,121.919548,41.619246,-3.132588e+06
1995-06-30,77418,1.652420e+06,3.087717,3.773010e+05,0.913639,0.768163,7.564375,35.965321,13.885897,-4.861862e+06
1996-06-30,77418,3.935138e+06,1.988631,6.490840e+05,0.392808,-0.050407,18.249801,16.001558,9.100687,-5.258027e+06
1997-06-30,77418,5.443073e+06,1.271430,4.398232e+05,0.111768,2.520221,13.598321,4.597881,4.922155,-1.725055e+06
1998-06-30,77418,2.272824e+07,3.822978,1.919487e+06,0.352648,2.969190,2.551360,0.903554,1.772812,-1.483403e+06
1999-06-30,77418,1.195158e+08,4.185493,2.438690e+07,1.072978,-0.133864,-0.379197,-0.537914,0.498832,1.222192e+07
2000-06-30,77418,1.216507e+08,0.957955,7.159994e+06,0.059908,0.058645,-0.363463,-0.549944,0.478177,3.736253e+06


In [54]:
100 * sample.wt1 / (sample.mv.shift(1) * sample.retx_p1)

date
1992-06-30           NaN
1993-06-30   -126.068482
1994-06-30   -932.750227
1995-06-30   -381.286919
1996-06-30   -160.010383
1997-06-30    -34.478683
1998-06-30     -7.128745
1999-06-30     12.847754
2000-06-30      3.263368
2001-12-31     11.138355
2002-12-31      0.343744
2003-12-31      0.443937
2004-12-31      0.430816
2005-12-31      0.777161
2006-12-31     -1.785537
2007-12-31      3.256074
2008-12-31      0.436943
2009-12-31     10.355815
2010-12-31      2.590956
2011-12-31      2.962464
2012-12-31      0.710306
2013-12-31           NaN
dtype: float64